# Opdracht oefenen met Pandas (dataset 'Wildlife Airstrikes')

## Het bestand *wildlife_strikes.csv* bevat data over dieren die in aanraking zijn gekomen met vliegtuigen. Dit is echte data die afkomstig is uit rapporten van de Amerikaanse Federal Aviation Association. Schrijf Python code om de volgende vragen te beantwoorden:

1) Laad het bestand in als DataFrame en toon de eerste 5 rijen van de tabel

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
strikes = pd.read_csv('wildlife_strikes.csv')
strikes.head(5)

2) Geef de vluchten weer waarbij mensen om het leven zijn gekomen.Toon de volgende kolommen:

  * Incident Year
  * Operator
  * Aircraft
  * Airport
  * Flight Phase
  * Height
  * Speed
  * Species Name
  * Flight Impact
  * Fatalities

In [ ]:
columns = ['Incident Year', 'Operator', 'Aircraft', 'Airport', 'Flight Phase', 
           'Height', 'Speed', 'Species Name', 'Flight Impact', 'Fatalities']

strikes.query("Fatalities > 0")[columns]

# of:
# strikes[strikes['Fatalities']>0][columns]

3) Gebruik matplotlib om een lijngrafiek te maken die het aantal incidenten per jaar toont. Op de x-as staan de jaartallen en op de y-as het aantal incidenten.

In [ ]:
strikes.groupby('Incident Year').size().plot()
plt.ylabel('Number of incidents')
plt.show()

4) Wat is de hoogste snelheid waarmee een vliegtuig een dier heeft geraakt? Toon van dit incident de volgende kolommen:

  * Incident Year
  * Operator
  * Aircraft
  * Speed
  * Species Name

In [ ]:
columns = ['Incident Year', 'Operator', 'Aircraft', 'Speed', 'Species Name']
strikes.query("Speed == Speed.max()")[columns]

# of:
# strikes[strikes['Speed'] == strikes['Speed'].max()][columns]

5) Wat is de gemiddelde hoogte waarop de incidenten plaatsvinden? Laat bij je berekening alle incidenten die op de grond plaatsvinden buiten beschouwing.

In [ ]:
strikes.query("Height != 0")['Height'].mean()

# of:
# strikes[strikes['Height']!=0]['Height'].mean()

6) Toon in een staafgrafiek het aantal incidenten per maand. Op de x-as staan dus de maanden, en op de y-as het aantal incidenten.

In [ ]:
import calendar
strikes.groupby('Incident Month').size().plot.bar()
plt.ylabel('Total no. of incidents')
plt.xticks(np.arange(12), calendar.month_abbr[1:13])
plt.show()

7) Vinden incidenten 's nachts op een andere hoogte plaats dan overdag? Toon in een staafgrafiek de gemiddelde hoogte waarop incidenten plaatsvonden, uitgesplitst op de kolom 'Visibility'. Op de x-as staat dus de visibility en op de y-as de gemiddelde hoogte.

In [ ]:
strikes.groupby('Visibility')['Height'].mean().plot.bar()
plt.show()

8) Toon alle diersoorten uit de tabel netjes onder elkaar, waarbij ieder dier maar één keer mag voorkomen. Sorteer op alfabetische volgorde.

In [ ]:
uniek = strikes['Species Name'].sort_values().unique()
mask = pd.notnull(uniek)  #Om de regel met NaN te verwijderen
uniek = uniek[mask]       #Om de regel met NaN te verwijderen
pd.DataFrame(uniek)

Het bestand 'birds.txt' bevat Engelse namen van bijna 1000 vogelsoorten. Gebruik dit bestand om de volgende vraag te kunnen maken.

9) Toon alle dieren uit de dataset, die géén vogels zijn. Toon twee kolommen: de naam van het dier en hoe vaak hij voorkomt in de dataset. Sorteer op de tweede kolom (van hoog naar laag).

In [ ]:
birds = np.loadtxt('birds.txt', dtype=str, delimiter=",")

not_birds = strikes[(~strikes['Species Name'].isin(birds)) &   # ipv ~ kan ook ==False
        (strikes['Species Name'].str.contains('BIRD')==False)]

# of:
# notnull = strikes[pd.notnull(strikes['Species Name'])]
# not_birds = notnull.query("`Species Name` not in @birds and \
#                            not `Species Name`.str.contains('BIRD')")

#Om alle output te laten zien kun je de volgende coderegel gebruiken:
#pd.set_option('display.max_rows', None)
#En om de optie weer uit te zetten kun je later in de notebook de volgende regel gebruiken:
#pd.reset_option('display.max_rows')

not_birds.groupby('Species Name').size().sort_values(ascending=False)

10) Er zijn veertien kolommen die aangeven of een onderdeel van het vliegtuig beschadigd is geraakt (van *Radome Damage* t/m *Other Damage*). Laat zien bij welke incidenten het grootste aantal onderdelen van het vliegtuig beschadigd zijn geraakt. Toon de kolommen *Aircraft*, *Species Name* en alle veertien damage kolommen.

In [ ]:
damage_columns = ['Radome Damage', 'Windshield Damage', 'Nose Damage', 'Engine1 Damage', 'Engine2 Damage', 
           'Engine3 Damage', 'Engine4 Damage', 'Propeller Damage', 'Wing or Rotor Damage', 
           'Fuselage Damage', 'Landing Gear Damage', 'Tail Damage', 'Lights Damage', 'Other Damage']
show_columns = ['Aircraft', 'Species Name'] + damage_columns
sum_damage = strikes[damage_columns].sum(axis = 1)
strikes[sum_damage == sum_damage.max()][show_columns]
# of:
# strikes.query("@sum_damage == @sum_damage.max()")[show_columns]

11) Toon van alle diersoorten die in de dataset voorkomen de volgende gegevens:
* de maximale hoogte waarop een individu van de soort in aanraking is gekomen met een vliegtuig
* de minimale hoogte
* de gemiddelde hoogte
* het aantal incidenten waarbij deze diersoort betrokken was

Zorg dat je alle incidenten die op de grond (height=0) plaatsvinden niet meeneemt. Sorteer op de maximale hoogte (van hoog naar laag).

De geoorde fuut (eared grebe, *Podiceps nigricollis*) vliegt volgens de IUCN (International Union for Conservation of Nature) op een maximale hoogte van 2500 meter ([link](http://www.iucnredlist.org/details/22696610/0)). De *wildlife strikes* dataset toont de hoogte in *feet*. Geeft deze dataset reden om ons beeld van deze fuut bij te stellen?

In [ ]:
pd.set_option('display.max_rows', None)
grouped_by_species = strikes.query("Height > 0").groupby('Species Name')
# of: 
# grouped_by_species = strikes[strikes['Height']>0].groupby('Species Name')
grouped_by_species['Height'].agg(['max', 'min', 'mean', 'count']).sort_values(by='max', ascending=False)

#EARED GREBE: 20000 feet = 6096 meter. Dus veel hoger dan wat bekend is.

12) Hoeveel incidenten hebben plaatsgevonden waarbij een dier betrokken was dat begint met de letter 'Z'?

In [ ]:
notnull = strikes[pd.notnull(strikes['Species Name'])]
len(notnull[notnull['Species Name'].str.startswith("Z")])